import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical

In [1]:
import pandas

In [2]:
df = pandas.read_csv("C:\\Users\\sante\\Desktop\\мой нлп - Лист1-2.csv", encoding="utf-8")

# тестовые данные
valid = df["title"].isin(['Интерстеллар', "Омерзительная восьмерка", "Тройной форсаж: Токийский дрифт"])
test = df[valid]
del test['title']

# удалили наши данные(оставили данные для обучения)
df = df.loc[df['title'] != "Интерстеллар"]
df = df.loc[df['title'] != "Омерзительная восьмерка"]
df = df.loc[df['title'] != "Тройной форсаж: Токийский дрифт"]
del df['title']

In [3]:
 df.head

<bound method NDFrame.head of                                                   text  label
0    Дон Блут – один из самых известных анимационны...     -1
1    Отрицательных рецензий на этот мультфильм очен...     -1
2    \nЧестно признаюсь: в детстве я если и смотрел...     -1
3    \nВпервые я познакомилась с творением студии '...     -1
4    \nЯ узнала про мультфильм 'Анастасия' на продл...     -1
..                                                 ...    ...
585  Питер Джексон уже не тот...  Пишу как человек,...     -1
586  Ждали-ждали - дождались  Около двух лет назад ...     -1
587  Три часа скукоты  В том, что «Хоббит: Нежданно...     -1
588  что-то случилось с любимым нами Средиземьем  В...     -1
589  Детям фильм смотреть нельзя  Фильм не понравил...     -1

[590 rows x 2 columns]>

In [4]:
def tokenizeWord(word):
    word = tokenizer.tokenize(word)
    if len(word) == 1:
        word = word[0]
    else:
        word = ""
    return word


In [5]:

import pymorphy2
from nltk.tokenize import RegexpTokenizer
morph = pymorphy2.MorphAnalyzer()
tokenizer = RegexpTokenizer(r'\w+')
uniqWords = []
tupleIndex = 0
sentences = []
for tuple in df.values:
    tupleIndex+=1
    words = tuple[0].split()
    wordsList = []
    for word in words:
        wordNF = tokenizeWord(morph.parse(word)[0].normal_form)
        uniqWords.append(wordNF)
        wordsList.append(wordNF)
    sentences.append(wordsList)
tupleIndex = 0
testSentences = []
for tuple in test.values:
    tupleIndex+=1
    words = tuple[0].split()
    wordsList = []
    for word in words:
        wordNF = tokenizeWord(morph.parse(word)[0].normal_form)
        uniqWords.append(wordNF)
        wordsList.append(wordNF)
    testSentences.append(wordsList)

In [27]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

print(len(uniqWords))

tokenizer = Tokenizer(num_words=len(uniqWords))
tokenizer.fit_on_texts(sentences)

vocab_size = len(tokenizer.word_index) + 1
maxlen = 100

X_train = tokenizer.texts_to_sequences(sentences)
X_test = tokenizer.texts_to_sequences(testSentences)

273477


In [28]:
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [29]:
print(vocab_size)


25916


In [30]:
from keras.utils import to_categorical
y_train = df["label"]
y_test = test["label"]

num_classes = 3

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


In [31]:
import numpy as np
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath, encoding="utf-8") as f:
        for line in f:
            word, *vector = line.split()
            word = word.split("_")[0]
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [32]:
embedding_dim = 300
embedding_matrix = create_embedding_matrix('C:/Users/sante/Desktop/model.txt', tokenizer.word_index, embedding_dim)

In [33]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
print(embedding_matrix.shape)
print(nonzero_elements)
nonzero_elements / vocab_size

(25916, 300)
14052


0.5422133045223029

In [37]:
from keras.layers import Dense, Dropout, Activation, Input, LSTM
from keras.models import Sequential
from keras import layers
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=True))
model.add(LSTM(300, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='sigmoid'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 300)          7774800   
_________________________________________________________________
lstm_5 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 903       
Total params: 8,496,903
Trainable params: 8,496,903
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [40]:
model.fit(X_train, y_train, epochs=30, batch_size=128)
predictions = model.predict(X_test)
print(classification_report(y_test.argmax(axis=1), predictions.argmax(axis=1)))

Epoch 1/30
590/590 [==============================] - 10s 17ms/step - loss: 0.2698 - accuracy: 0.9220
Epoch 2/30
590/590 [==============================] - 10s 17ms/step - loss: 0.2346 - accuracy: 0.9339
Epoch 3/30
590/590 [==============================] - 10s 16ms/step - loss: 0.1924 - accuracy: 0.9458
Epoch 4/30
590/590 [==============================] - 10s 17ms/step - loss: 0.1552 - accuracy: 0.9593
Epoch 5/30
590/590 [==============================] - 10s 16ms/step - loss: 0.1514 - accuracy: 0.9661
Epoch 6/30
590/590 [==============================] - 10s 17ms/step - loss: 0.1591 - accuracy: 0.9661
Epoch 7/30
590/590 [==============================] - 10s 17ms/step - loss: 0.1630 - accuracy: 0.9525
Epoch 8/30
590/590 [==============================] - 10s 16ms/step - loss: 0.1105 - accuracy: 0.9814
Epoch 9/30
590/590 [==============================] - 10s 17ms/step - loss: 0.0805 - accuracy: 0.9847
Epoch 10/30
590/590 [==============================] - 10s 17ms/step - loss: 0.079